In [ ]:
from github import Github, enable_console_debug_logging
import pandas as pd
from tqdm import tqdm
from IPython.display import clear_output
import pathlib
import base64
import os
repo_dir = '../../BigInput/repos2'
login = 'Miejsce na login'
haslo = 'Miejsce na haslo'
query = 'data science portfolio'
language = 'Jupyter Notebook'
allowed_extensions = ['md', 'ipynb', 'py', 'sh']
attributes = ['full_name', 'clone_url', 'description', 'created_at']
#lista dostępnych atrybutów
#https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html

g = Github(login, haslo)

In [ ]:
result = g.search_repositories(query, sort='stars', order='desc')

In [ ]:
jupyter_repo = [e for e in result if e.language == language]

In [ ]:
def get_attributes(l, attributes):
    result = []
    for e in l:
        d = dict()
        for attr in attributes:
            d[attr] = getattr(e, attr)
        result.append(d)
    return result


df = pd.DataFrame(get_attributes(jupyter_repo, attributes))

In [ ]:
if not os.path.exists(repo_dir):
    os.mkdir(repo_dir)

In [ ]:
df.to_csv(os.path.join(repo_dir, 'data.csv'), index=False)

In [ ]:
from tqdm import tqdm
import requests
from time import sleep
from datetime import datetime


def download_file(url, path, name):
    response = requests.get(url, stream=True)
    pathlib.Path(path).mkdir(parents=True, exist_ok=True)
    file_path = os.path.join(path, name)
    print(file_path)
    with open(file_path, "wb") as handle:
        for data in response.iter_content():
            handle.write(data)

In [ ]:
def check_limit(g):
    limit = g.get_rate_limit().rate.remaining
    if limit < 10:
        print('Limit reached')
        while limit < 10:
            sleep(60)
            limit = g.get_rate_limit().rate.remaining
        print('New limit')

In [ ]:
def save_error(e, err, repo_dir):
    with open(os.path.join(repo_dir, 'errors.txt'), 'a') as f:
        f.write(e.full_name + ': ' + str(datetime.now()))
        f.write('\n')
        f.write(str(err))
        f.write('\n')

In [ ]:
def download(g, repo, allowed_extensions, local_path, git_path='/'):
    check_limit(g)
    contents = repo.get_dir_contents(git_path)
    for e in contents:
        if e.type == 'dir' and e.name != '.ipynb_checkpoints'\
            and e.name !='node_modules':
            download(g, repo, allowed_extensions, local_path, e.path)
        else:
            name = e.name
            if name.split('.')[-1] in allowed_extensions:
                if git_path != '/':
                    path = os.path.join(local_path, git_path)
                else:
                    path = local_path
                download_file(e.download_url, path, name)


for e in tqdm(jupyter_repo):
    try:
        local_path = os.path.join(repo_dir, e.full_name)
        download(g, e, allowed_extensions, local_path)
        clear_output(wait=True)
    except Exception as err:
        save_error(e, err, repo_dir)